In [6]:
# 載入需要的套件
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import sys
import datetime
from bs4 import BeautifulSoup

options = Options()
options.add_argument("--disable-notifications")

In [ ]:
# a=input()

In [ ]:
website = 'https://web.pcc.gov.tw'

In [ ]:
try:
    # 開啟瀏覽器視窗(Chrome)
    # 方法一：執行前需開啟chromedriver.exe且與執行檔在同一個工作目錄
    driver = webdriver.Chrome(chrome_options=options)
    driver.maximize_window()
    # 方法二：或是直接指定exe檔案路徑
    # driver = webdriver.Chrome(“桌面\chromedriver”)
    driver.get("https://web.pcc.gov.tw/prkms/tender/common/basic/indexTenderBasic") # 更改網址以前往不同網頁
    time.sleep(2)
#     driver.set_window_size(1920, 1080)
    # time.sleep(3)
except:
#     Warning('開啟瀏覽器失敗!')
    sys.exit()

C:\Users\GeorgeLin\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [ ]:
input('完成查詢後按Enter繼續')

完成查詢後按Enter繼續


''

# 調整每頁顯示筆數

In [ ]:
# 調整為每頁顯示100筆
pagesize = driver.find_element_by_id("pageSizeSelector")
pagesize.send_keys('100')

# 撈取清單

In [ ]:
# soup = BeautifulSoup(driver.page_source, "html.parser")
# next_page = soup.find('a', text = '下一頁')
# next_page
# BeautifulSoup(driver.page_source, "html.parser").find('a', text = '下一頁')

In [ ]:
def get_case_list(website=website):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # soup
    table = soup.find('table', {'class': 'tb_01'})
    # table

    columns = [th.text.replace('\n', '').replace('\u2003', '') for th in table.find('tr').find_all('th')]
    columns

    trs = table.find_all('tr')[1:]
    rows = list()
    for tr in trs: #.replace('\xa0', '')
        rows.append([td.text.replace('\n', '').replace('\t', '').replace('!', '').replace(' ', '') for td in tr.find_all('td')])


    links = table.find_all(["div"], class_ = 'bt_cen1 no_blank')
    link_li = []
    for link in links:
        link_li.append(website+link.select_one("a").get("href"))

    df = pd.DataFrame(data = rows, columns = columns)
    df['連結'] = link_li
    time.sleep(1)
    return df 
    # df

In [ ]:
try:
    df = get_case_list()
    while BeautifulSoup(driver.page_source, "html.parser").find('a', text = '下一頁') != None:
        driver.find_element_by_link_text( "下一頁").click()
        time.sleep(1)
        df = pd.concat([df, get_case_list()])
    print('共有{}筆資料'.format(len(df)))
    df
except:
    driver.quit()
    sys.exit()

共有1238筆資料


In [ ]:
# driver.find_elements_by_xpath('//*[contains(text(), "下一頁")]')

[<selenium.webdriver.remote.webelement.WebElement (session="008b3abf0c2d2516216b04cdcd37e9a7", element="3e82460f-a7bb-480a-8100-f51499160c5d")>]

# 撈取聯絡資訊

In [ ]:
# driver = webdriver.Chrome()

In [ ]:
def get_contact_info(link):
#     wait = WebDriverWait(driver,30)
#     driver = webdriver.Chrome()
    driver.get(link) # 更改網址以前往不同網頁
    while len(driver.find_elements_by_xpath('//*[contains(text(), "驗證碼檢核")]')) != 0:
        time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find('table', {'summary': '機關資料'})
    trs = table.find_all('tr')[1:]
    rows = list()
    for tr in trs:
        rows.append([td.text.replace('\n', '').replace('\t', '').replace('!', '').replace(' ', '') for td in tr.find_all('td')])
    info = pd.DataFrame(rows).set_index([0]).T
    info['連結'] = link
    time.sleep(1)
#     driver.quit()
    return info

In [ ]:
try:
    info = pd.DataFrame()
    for i, link in enumerate(df['連結']):
    #     print(i)
        info = pd.concat([info, get_contact_info(link)])

except:
    # 避免中途中止
    result = pd.merge(df, info, on = '連結', how = 'left')
    result.to_excel('./政府電子採購網聯絡資訊{:%m%d}.xlsx'.format(datetime.datetime.today()), index = False)
#     result
    driver.quit()
    sys.exit()

In [ ]:
# 輸出
result = pd.merge(df, info, on = '連結', how = 'left')
result.to_excel('./政府電子採購網聯絡資訊{:%m%d}.xlsx'.format(datetime.datetime.today()), index = False)
result

,項次,機關名稱,標案案號標案名稱,傳輸次數,招標方式,採購性質,公告日期,截止投標,預算金額,功能選項,連結,機關代碼,機關名稱,單位名稱,機關地址,聯絡人,聯絡電話,傳真號碼,電子郵件信箱
0,1,桃園市政府水務局,1111005-Y1(更正公告)桃園市雨水下水道即時水情監測系統建置計畫,01,經公開評選或公開徵求之限制性招標,勞務類,111/10/07,111/11/07,"54,243,000",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.80.25,桃園市政府水務局,雨水下水道科,330桃園市桃園區成功路1段32號7樓,高振傑,(03)3033688#3736,(03)3033662,086025@mail.tycg.gov.tw
1,2,臺東縣東河鄉公所,TH111018(更正公告)農東河203及206農路改善工程,01,公開招標,工程類,111/10/07,111/10/18,"4,374,000",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.76.54.57,臺東縣東河鄉公所,臺東縣東河鄉公所,959臺東縣東河鄉東河村11鄰311號,許晉瑞,(089)896200#239,(089)896378,dhe003@dh.taitung.gov.tw
2,3,財團法人國家衛生研究院,111B0C0239(更正公告)111年度個人電腦暨週邊設備維護服務案一式,01,經公開評選或公開徵求之限制性招標,勞務類,111/10/07,111/10/19,"3,500,000",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,92019239,財團法人國家衛生研究院,財團法人國家衛生研究院,350苗栗縣竹南鎮科研路35號,採購中心蔡懿芳/規格:陳財通(32803),(037)206166#32708,(037)587405,voic@nhri.edu.tw
3,4,國立嘉義大學,11101084(更正公告)手持式土壤溫室氣體偵測儀,02,公開招標,財物類,111/10/07,111/10/12,"1,550,000",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.10.35,國立嘉義大學,事務組,600嘉義市東區鹿寮里學府路300號,蘇美蓉,(05)2717112,(05)2717115,renee@mail.ncyu.edu.tw
4,5,台灣中油股份有限公司,T1411F042(更正公告)潤滑油添加劑CE012(月桂醇二聚氧乙烯醚硫酸鈉)一批,03,公開招標,財物類,111/10/07,111/10/17,,檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.13.50,台灣中油股份有限公司,採購處南部採購中心採購三組,811高雄市楠梓區左楠路2號,盧鈺雅,(07)5824141#2773,(07)5823230,708593@cpc.com.tw
5,6,台灣中油股份有限公司探採事業部,BEB1100006(更正公告)探採事業部產品碳足跡盤查輔導暨查證工作,02,公開招標,勞務類,111/10/07,111/10/12,"2,940,000",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.13.50.50,台灣中油股份有限公司探採事業部,台灣中油股份有限公司探採事業部,360苗栗縣苗栗市中正路140號1F,阮郁奇先生,(037)262100#385,(037)273950,355585@cpc.com.tw
6,7,基隆市暖暖區碇內國民小學,111A019(更正公告)幼兒園教學環境改善,01,公開取得報價單或企劃書,工程類,111/10/07,111/10/12,"953,972",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.76.57.97.33,基隆市暖暖區碇內國民小學,基隆市暖暖區碇內國民小學,205基隆市暖暖區源遠路258號,林三助,(02)24581300#30,(02)24585831,san19680110@yahoo.com.tw
7,8,行政院,K111100601行政院院區員工福利餐廳提供膳食委外辦理案,01,公開取得報價單或企劃書,勞務類,111/10/06,111/10/13,"697,815",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3,行政院,行政院秘書處,100臺北市中正區忠孝東路一段一號,吳小姐,(02)33566959,(02)33566966,wuyawen@ey.gov.tw
8,9,臺灣菸酒股份有限公司埔里酒廠,"111-0380-1-025微香酒蛋手提紙盒30,000個",01,公開取得報價單或企劃書,財物類,111/10/06,111/10/13,,檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.7.74.35,臺灣菸酒股份有限公司埔里酒廠,臺灣菸酒股份有限公司埔里酒廠,545南投縣埔里鎮中山路3段219號,陳宏明,(049)2984006#431,(049)2981861,080199@mail.ttl.com.tw
9,10,臺北榮民總醫院蘇澳分院,sa111323E化行動護理車2台,01,公開取得報價單或企劃書,財物類,111/10/06,111/10/12,"244,000",檢視,https://web.pcc.gov.tw/prkms/urlSelector/commo...,3.32.80.4,臺北榮民總醫院蘇澳分院,採購工作小組,270宜蘭縣蘇澳鎮蘇濱路一段301號,周美琴,(03)9222141#2210,(03)9225125,ysvhb0055@mail.ysvh.gov.tw


In [ ]:
driver.quit()

In [ ]:
# driver.get(df['連結'][0]) # 更改網址以前往不同網頁
# soup = BeautifulSoup(driver.page_source, "html.parser")
# # soup
# table = soup.find('table', {'summary': '機關資料'})
# # table
# trs = table.find_all('tr')[1:]
# rows = list()
# for tr in trs: #.replace('\xa0', '')
#     rows.append([td.text.replace('\n', '').replace('\t', '').replace('!', '').replace(' ', '') for td in tr.find_all('td')])

In [ ]:
# t = pd.DataFrame(rows).set_index([0]).T
# t

,機關代碼,機關名稱,單位名稱,機關地址,聯絡人,聯絡電話,傳真號碼,電子郵件信箱
1,3.80.25,桃園市政府水務局,雨水下水道科,330桃園市桃園區成功路1段32號7樓,高振傑,(03)3033688#3736,(03)3033662,086025@mail.tycg.gov.tw
